In [52]:
import webbrowser
from bs4 import BeautifulSoup, NavigableString, Tag
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
import  os
import pandas as pd
import json
import bs4
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from urllib3 import request
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.select import Select
import datetime
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import numpy as np
################ JUST MAKE CHANGES HERE ########################
startDate = '2021/06/01'  # <- FORMAT (YYYY/MM/DD)
endDate = '2021/06/30'
symbol = 'VTI'
################################################################

############### RESULTING FILE FORMAT ###########################

#           '<SYMBOL>_<DD>_<MM>_<YYYY>.CSV'

#################################################################

cols = ['Date (DD/MM/YYYY)','Broker','Buy Value ($)','Sell Value ($)','Net Value ($)','Total Value ($)','Buy Volume','Sell Volume','Net Volume','Total Volume','Buy Trade Count','Sell Trade Count','Total Trade Count','Buy Price','Sell Price','Rank']
graph1_df = pd.DataFrame(columns=cols)
graph2_df = pd.DataFrame(columns=cols)
file_name = ''

def workdays(d, end, excluded=(6, 7)):
    days = []
    while d.date() <= end.date():
        if d.isoweekday() not in excluded:
            days.append(d.strftime('%Y/%m/%d'))
        d += datetime.timedelta(days=1)
    return days

def main():

    global startDate
    global endDate
    global symbol
    global graph1_df
    global graph2_df
    global file_name
    cols = ['Date (DD/MM/YYYY)','Broker','Buy Value ($)','Sell Value ($)','Net Value ($)','Total Value ($)','Buy Volume','Sell Volume','Net Volume','Total Volume','Buy Trade Count','Sell Trade Count','Total Trade Count','Buy Price','Sell Price','Rank']
    

    prefs = {"profile.default_content_setting_values.notifications" : 2}  #block notifications
    chrome_options = Options()
    chrome_options.add_experimental_option("prefs",prefs)
    chrome_options.add_argument("--no-sandbox")
    # chrome_options.add_argument("--headless")     
    #chrome_options.add_argument("--disable-setuid-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    #chrome_options.add_argument('--disable-features=VizDisplayCompositor')
    driver = webdriver.Chrome(ChromeDriverManager().install(),options=chrome_options)

    driver.get("https://scanner.daytradescans.com/signin")
    driver.implicitly_wait(20)
    driver.maximize_window()

    user_name='//*[@id="user_email"]'
    password='//*[@id="user_password"]'
    submit_button='//*[@id="new_user"]/div[2]/div/input'
    fundamentals='//*[@id="navbarDropdown"]'
    broker_data = '//*[@id="navbarSupportedContent"]/ul/li[1]/div/a[2]'
    capture = '//*[@id="capture-broker-data"]/span'
    dl = '//*[@id="canvas-link"]'

    #on_login_page
    driver.find_element_by_xpath(user_name).send_keys('ivan.cortez.paulo@gmail.com')  #login_id
    driver.find_element_by_xpath(password).send_keys("qwertyuiop2021")    #password
    driver.find_element_by_xpath(submit_button).click()
    driver.implicitly_wait(30)

    print("logged in")

    driver.find_element_by_xpath(fundamentals).click()
    driver.find_element_by_xpath(broker_data).click()

    print("in broker data")
    

    startDate = datetime.datetime.strptime(startDate, "%Y/%m/%d")
    endDate = datetime.datetime.strptime(endDate, "%Y/%m/%d")
    
    # driver.find_element_by_xpath(text_box).send_keys(input_for_search)
    # driver.find_element_by_xpath(from_date).send_keys(from_date_in)
    # driver.find_element_by_xpath(to_date).send_keys(to_date_in)
    # driver.find_element_by_xpath(search_button).click()

    import time
    
    dates = workdays(startDate,endDate)

    filedate1 = datetime.datetime.strptime(dates[0],'%Y/%m/%d').strftime('%d/%m/%Y').replace('/','_')
    filedate2 = datetime.datetime.strptime(dates[-1],'%Y/%m/%d').strftime('%d/%m/%Y').replace('/','_')

    for date in dates:

        new_url = f'https://scanner.daytradescans.com/brokers?code_autocomplete_label={symbol}&code=&name_autocomplete_label=&name=&from={date}&to={date}&commit=Search&minimum_share_price=&maximum_share_price=&minimum_market_cap=&maximum_market_cap=&minimum_eps=&maximum_eps=&minimum_soi=&maximum_soi='
        driver.get(new_url)
        driver.implicitly_wait(10)

        driver.find_element_by_xpath(capture).click()
        driver.find_element_by_xpath(dl).click()
        
        driver.back()
        html=driver.page_source
        soup = BeautifulSoup(html,features="lxml")
        soup_data = soup.get_text()
        a = json.loads(soup_data)

        if bool(a):
            df = pd.DataFrame(a)
            df.drop(['id'],axis=1, inplace = True)
            # import pdb; pdb.set_trace()
            df['buy_volume'] = df['buy_volume'].map(float)
            df['sell_volume'] = df['sell_volume'].map(float)

            df['total_volume'] = abs(df['buy_volume']) + abs(df['sell_volume'])
            tsum = df.tail(1)['total_volume']
            df['Rank'] = df['total_volume']
            df.Rank = df.Rank.div(float(tsum)) * 100
            df.Rank = df.Rank.round(2)

            td = datetime.datetime.strptime(date,'%Y/%m/%d').strftime('%d/%m/%Y')
            df.rename(columns={'name':'Broker','buy_value':'Buy Value ($)','sell_value':'Sell Value ($)','net_value':'Net Value ($)','total_value':'Total Value ($)','buy_volume':'Buy Volume','sell_volume':'Sell Volume','net_volume':'Net Volume','total_volume':'Total Volume','buy_trade_count':'Buy Trade Count','sell_trade_count':'Sell Trade Count','total_trade_count':'Total Trade Count','buy_price':'Buy Price','sell_price':'Sell Price'},inplace=True)
            df['Date (DD/MM/YYYY)'] = td

            df = df[cols]

            file_name = f'{symbol}_{filedate1}_to_{filedate2}.csv'
            if not(os.path.exists(file_name) and os.path.getsize(file_name) > 0):
                df.to_csv(file_name, mode = 'a',index=False)
            else:
                df.to_csv(file_name, mode = 'a', header = False, index=False)
#             import pdb; pdb.set_trace()
            graph1_df = graph1_df.append(df.tail(1))
        
    time.sleep(10)
    driver.close()


In [53]:
file_name = 'VTI_01_06_2021_to_30_06_2021.csv'

In [54]:
def graph1_plotting():
    global file_name
    
    df = pd.read_csv(file_name)
    
    df = df[df['Broker'] == 'Total']
    
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.03, subplot_titles=('High-Low', 'Volume'), row_width=[0.2, 0.7])

    fig.add_trace(go.Candlestick(x=df['Date (DD/MM/YYYY)'], open=df['Sell Price'], high=df['Sell Price'], low=df['Buy Price'], close=df['Buy Price']),row=1, col=1)
    
    fig.add_trace(go.Bar(x=df['Date (DD/MM/YYYY)'], y=df['Total Volume'] ,showlegend=False), row=2, col=1)

    fig.update(layout_xaxis_rangeslider_visible=False)
    
    fig.show()
graph1_plotting()

In [55]:
def volume_plotting():
    global file_name
    df = pd.read_csv(file_name)
    
    broker_df = df[df['Broker'] != 'Total']
    
    fig = px.bar(broker_df, x="Date (DD/MM/YYYY)", y="Total Volume", color="Broker", hover_data=['Date (DD/MM/YYYY)'], barmode = 'stack')
    
    fig.show()
volume_plotting()

In [56]:
pd.options.mode.chained_assignment = None  # default='warn'
 
def process_df(df):
    df['Sell Price'] = np.where(df['Sell Price'] == 0,df['Buy Price'],df['Sell Price'])
    df['Buy Price'] = np.where(df['Buy Price'] == 0,df['Sell Price'],df['Buy Price'])

def overlay_plotting():
    global file_name
    
    graph1_df = pd.read_csv(file_name)
    df = graph1_df[graph1_df['Broker'] != 'Total']
    lob = df['Broker'].unique().tolist()
    
    process_df(df)
    
    fig = make_subplots(rows=2, cols=1, shared_xaxes=False, vertical_spacing=0.03, subplot_titles=('High-Low',''), row_width=[0.2, 0.7])
    for i in lob:
        temp_df = df[df['Broker'] == i]
        fig.add_trace(go.Candlestick(x=temp_df['Date (DD/MM/YYYY)'], open=temp_df['Sell Price'], high=temp_df['Sell Price'], low=temp_df['Buy Price'], close=temp_df['Buy Price'], name=i, visible='legendonly'),row=1, col=1)

    fig.update(layout_xaxis_rangeslider_visible=False)

    fig.show()

overlay_plotting()

# Net Code

In [60]:
import webbrowser
from bs4 import BeautifulSoup, NavigableString, Tag
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
import os
import sys
import pandas as pd
import json
import bs4
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from urllib3 import request
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.select import Select
import datetime
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import numpy as np
################ JUST MAKE CHANGES HERE ########################
startDate = '2021/07/01'  # <- FORMAT (YYYY/MM/DD)
endDate = '2021/07/15'
symbol = 'VTI'
################################################################

############### RESULTING FILE FORMAT ###########################

#           '<SYMBOL>_<start_date>_to_<end_date>.CSV'

#################################################################

def workdays(d, end, excluded=(6, 7)):
    days = []
    while d.date() <= end.date():
        if d.isoweekday() not in excluded:
            days.append(d.strftime('%Y/%m/%d'))
        d += datetime.timedelta(days=1)
    return days



cols = ['Date (DD/MM/YYYY)','Broker','Buy Value ($)','Sell Value ($)','Net Value ($)','Total Value ($)','Buy Volume','Sell Volume','Net Volume','Total Volume','Buy Trade Count','Sell Trade Count','Total Trade Count','Buy Price','Sell Price','Rank']
graph1_df = pd.DataFrame(columns=cols)

startDate = datetime.datetime.strptime(startDate, "%Y/%m/%d")
endDate = datetime.datetime.strptime(endDate, "%Y/%m/%d")
dates = workdays(startDate,endDate)

filedate1 = datetime.datetime.strptime(dates[0],'%Y/%m/%d').strftime('%d/%m/%Y').replace('/','_')
filedate2 = datetime.datetime.strptime(dates[-1],'%Y/%m/%d').strftime('%d/%m/%Y').replace('/','_')
file_name = f'{symbol}_{filedate1}_to_{filedate2}.csv'

def main():

    global symbol
    global graph1_df
    global file_name
    global dates

    cols = ['Date (DD/MM/YYYY)','Broker','Buy Value ($)','Sell Value ($)','Net Value ($)','Total Value ($)','Buy Volume','Sell Volume','Net Volume','Total Volume','Buy Trade Count','Sell Trade Count','Total Trade Count','Buy Price','Sell Price','Rank']
    

    prefs = {"profile.default_content_setting_values.notifications" : 2}  #block notifications
    chrome_options = Options()
    chrome_options.add_experimental_option("prefs",prefs)
    chrome_options.add_argument("--no-sandbox")
    # chrome_options.add_argument("--headless")     
    #chrome_options.add_argument("--disable-setuid-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    #chrome_options.add_argument('--disable-features=VizDisplayCompositor')
    driver = webdriver.Chrome(ChromeDriverManager().install(),options=chrome_options)

    driver.get("https://scanner.daytradescans.com/signin")
    driver.implicitly_wait(20)
    driver.maximize_window()

    user_name='//*[@id="user_email"]'
    password='//*[@id="user_password"]'
    submit_button='//*[@id="new_user"]/div[2]/div/input'
    fundamentals='//*[@id="navbarDropdown"]'
    broker_data = '//*[@id="navbarSupportedContent"]/ul/li[1]/div/a[2]'
    capture = '//*[@id="capture-broker-data"]/span'
    dl = '//*[@id="canvas-link"]'

    #on_login_page
    driver.find_element_by_xpath(user_name).send_keys('ivan.cortez.paulo@gmail.com')  #login_id
    driver.find_element_by_xpath(password).send_keys("qwertyuiop2021")    #password
    driver.find_element_by_xpath(submit_button).click()
    driver.implicitly_wait(30)

    print("logged in")

    driver.find_element_by_xpath(fundamentals).click()
    driver.find_element_by_xpath(broker_data).click()

    print("in broker data")
    
    # driver.find_element_by_xpath(text_box).send_keys(input_for_search)
    # driver.find_element_by_xpath(from_date).send_keys(from_date_in)
    # driver.find_element_by_xpath(to_date).send_keys(to_date_in)
    # driver.find_element_by_xpath(search_button).click()  

    for date in dates:

        new_url = f'https://scanner.daytradescans.com/brokers?code_autocomplete_label={symbol}&code=&name_autocomplete_label=&name=&from={date}&to={date}&commit=Search&minimum_share_price=&maximum_share_price=&minimum_market_cap=&maximum_market_cap=&minimum_eps=&maximum_eps=&minimum_soi=&maximum_soi='
        driver.get(new_url)
        driver.implicitly_wait(10)

        driver.find_element_by_xpath(capture).click()
        driver.find_element_by_xpath(dl).click()
        
        driver.back()
        html=driver.page_source
        soup = BeautifulSoup(html,features="lxml")
        soup_data = soup.get_text()
        a = json.loads(soup_data)

        if bool(a):
            df = pd.DataFrame(a)
            df.drop(['id'],axis=1, inplace = True)
            # import pdb; pdb.set_trace()
            df['buy_volume'] = df['buy_volume'].map(float)
            df['sell_volume'] = df['sell_volume'].map(float)

            df['total_volume'] = abs(df['buy_volume']) + abs(df['sell_volume'])
            tsum = df.tail(1)['total_volume']
            df['Rank'] = df['total_volume']
            df.Rank = df.Rank.div(float(tsum)) * 100
            df.Rank = df.Rank.round(2)

            td = datetime.datetime.strptime(date,'%Y/%m/%d').strftime('%d/%m/%Y')
            df.rename(columns={'name':'Broker','buy_value':'Buy Value ($)','sell_value':'Sell Value ($)','net_value':'Net Value ($)','total_value':'Total Value ($)','buy_volume':'Buy Volume','sell_volume':'Sell Volume','net_volume':'Net Volume','total_volume':'Total Volume','buy_trade_count':'Buy Trade Count','sell_trade_count':'Sell Trade Count','total_trade_count':'Total Trade Count','buy_price':'Buy Price','sell_price':'Sell Price'},inplace=True)
            df['Date (DD/MM/YYYY)'] = td

            df = df[cols]

            if not(os.path.exists(file_name) and os.path.getsize(file_name) > 0):
                df.to_csv(file_name, mode = 'a',index=False)
            else:
                df.to_csv(file_name, mode = 'a', header = False, index=False)
                
            graph1_df = graph1_df.append(df.tail(1))
        
    time.sleep(10)
    driver.close()

def graph1_plotting():
    global file_name
    
    df = pd.read_csv(file_name)
    
    df = df[df['Broker'] == 'Total']
    
    df['Date (DD/MM/YYYY)'] = pd.to_datetime(df['Date (DD/MM/YYYY)'],format='%d/%m/%Y')
    
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.03, subplot_titles=('High-Low', 'Volume'), row_width=[0.2, 0.7])

    fig.add_trace(go.Candlestick(x=df['Date (DD/MM/YYYY)'], open=df['Sell Price'], high=df['Sell Price'], low=df['Buy Price'], close=df['Buy Price']),row=1, col=1)
    
    fig.add_trace(go.Bar(x=df['Date (DD/MM/YYYY)'], y=df['Total Volume'] ,showlegend=False), row=2, col=1)

    fig.update(layout_xaxis_rangeslider_visible=False)
    
    fig.show()

def volume_plotting():
    global file_name
    df = pd.read_csv(file_name)
    
    df['Date (DD/MM/YYYY)'] = pd.to_datetime(df['Date (DD/MM/YYYY)'],format='%d/%m/%Y')
    
    broker_df = df[df['Broker'] != 'Total']
    
    fig = px.bar(broker_df, x="Date (DD/MM/YYYY)", y="Total Volume", color="Broker", barmode = 'stack',text="Broker",labels={
                     "Date (DD/MM/YYYY)": "Date"})
    fig.update_traces(hovertemplate='Broker = %{text} <br>Date = %{x} <br>Total Volume = %{y}')
    fig.update_traces(textposition=None)
    fig.show()

pd.options.mode.chained_assignment = None  # default='warn'

def process_df(df):
    df['Sell Price'] = np.where(df['Sell Price'] == 0,df['Buy Price'],df['Sell Price'])
    df['Buy Price'] = np.where(df['Buy Price'] == 0,df['Sell Price'],df['Buy Price'])

def overlay_plotting():
    global file_name
    
    graph1_df = pd.read_csv(file_name)
    df = graph1_df[graph1_df['Broker'] != 'Total']
    df['Date (DD/MM/YYYY)'] = pd.to_datetime(df['Date (DD/MM/YYYY)'],format='%d/%m/%Y')
    lob = df['Broker'].unique().tolist()
    
    process_df(df)
    
    fig = make_subplots(rows=2, cols=1, shared_xaxes=False, vertical_spacing=0.03, subplot_titles=('High-Low',''), row_width=[0.2, 0.7])
    for i in lob:
        temp_df = df[df['Broker'] == i]
        fig.add_trace(go.Candlestick(x=temp_df['Date (DD/MM/YYYY)'], open=temp_df['Sell Price'], high=temp_df['Sell Price'], low=temp_df['Buy Price'], close=temp_df['Buy Price'], name=i),row=1, col=1)

    fig.update(layout_xaxis_rangeslider_visible=False)

    fig.show()

# if __name__ == "__main__":
#     main()
#     choice = 1
#     while True:
        
#         while(True):
#             os.system('cls')
#             print('-------- PLOTTING - MENU -------- \n ')
#             print(' 1. Total Volume Graph')
#             print(' 2. All Brokers Volume Plotting Bar Chart')
#             print(' 3. All Brokers Open Close Low High Graph')
#             print(' 4. Exit')
#             choice = input(' Enter the choice : ')
#             if choice == '1':
#                 graph1_plotting()
#             elif choice == '2':
#                 volume_plotting()
#             elif choice == '3':
#                 overlay_plotting()
#             elif choice == '4':
#                 sys.exit('Exiting .. ')
#             else:
#                 print('Invalid Option Try Again')


In [61]:
file_name = 'VTI_01_06_2021_to_30_06_2021.csv'

In [62]:
volume_plotting()